TO DO:
- Create dict of tabs to reference in excel PSDs

#### Pseudo Code

- Convert original pricing PSD to dataframe
- Create dataframe or dictionary containing partnumber, SAP description, and new pricing data
- Iterate through and update pricing in dataframe referencing pricing dictionary/dataframe



- Add 2 columns: 1 to contain original pricing, and 1 to calculate percent difference?
- Write new tabs to excel
"""

#### Imports

In [1]:
import pandas as pd
import os
import openpyxl

#### Convert original pricing PSD to dataframe

In [2]:
# Folder where PSDs are located
fileDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\20-29 Areas\22 grundfos-express-tools\Price Updater\Input PSDs"

file_to_change = r"CR_StdPumps.xlsx"
filePath = os.path.join(fileDir, file_to_change)

# Create dataframe from PSD
# psd_data = pd.read_excel(filePath, sheet_name=1, header=1)
psd_data = pd.read_excel(filePath, sheet_name=1, header=None, na_filter=False)

# This removes the header data from the dataframe to avoid duplicating during excel write operation
# psd_data = psd_data.iloc[3::,:] 

In [3]:
new_header = psd_data.iloc[1] #grab the 2nd row for the header
psd_data.columns = new_header #set the header row as the df header

#### Create dataframe or dictionary containing partnumber, SAP description, and new pricing data

In [4]:
newDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\20-29 Areas\22 grundfos-express-tools\Price Updater"
newPrices = r"2022_Big_Book.xlsx"
pathNewPrices = os.path.join(newDir, newPrices)

# Create dataframe of new prices
cols = ['Part Number', '2022 List Prices']
newPrice_data = pd.read_excel(pathNewPrices, sheet_name=0, usecols=cols)

# Renaming part number column for use in pandas merge
newPrice_data.rename(columns = {'Part Number':'Big Book Part Number'}, inplace=True)

#### Reorder df column function

In [5]:
def reorder_columns(dataframe, col_name, position):
    """Reorder a dataframe's column.
    Args:
        dataframe (pd.DataFrame): dataframe to use
        col_name (string): column name to move
        position (0-indexed position): where to relocate column to
    Returns:
        pd.DataFrame: re-assigned dataframe
    """
    temp_col = dataframe[col_name]
    dataframe = dataframe.drop(columns=[col_name])
    dataframe.insert(loc=position, column=col_name, value=temp_col)
    return dataframe

#### Iterate through and update pricing in dataframe referencing pricing dictionary/dataframe

In [6]:
# Essentially a VLOOKUP in python
df = psd_data.merge(newPrice_data, how='left', left_on='PartNumber', right_on='Big Book Part Number')

In [7]:
# Find Column Index of "ListPrice" within excel file
insert_loc_col = df.columns.get_loc("ListPrice") + 1

# Re ordering columns for QA review. Insert New Columns (PN, List Price) next to "ListPrice" column for comparison
# df = reorder_columns(dataframe=df3, col_name='Big Book Part Number', position=last_col)
df = reorder_columns(dataframe=df, col_name='2022 List Prices', position=insert_loc_col)
df = reorder_columns(dataframe=df, col_name='Big Book Part Number', position=insert_loc_col)

In [65]:
# new_header = df.iloc[1] # grab the 2nd row for the header
# psd_data.columns = new_header # set the header row as the df header

#### Create Working Copy of pricing PSD; leaving original untouched

In [ ]:
import shutil

filename, file_ext = file_to_change.split(os.extsep)

outputDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\20-29 Areas\22 grundfos-express-tools\Price Updater\Output PSDs"
outputFilename = filename + " - Ready for Review." + file_ext
outputPath = os.path.join(outputDir, outputFilename)

shutil.copyfile(filePath, outputPath) # Create copy to leave original untouched

#### Write to new tab in excel sheet. Do we need to add these new prices for history tracking?

In [67]:
# Write dataframe 
with pd.ExcelWriter(outputPath, mode="a", engine="openpyxl", if_sheet_exists='replace') as writer:
    # df3.to_excel(writer, sheet_name="CP BpaQ", header=None, startrow=writer.sheets["Sheet1"].max_row, index=False)
    df.to_excel(writer, sheet_name="CR BpaQ", header=None, index=False)

#### Adds column headers to new pricing columns for historial archive purposes. Do we need to do this?

In [14]:
from openpyxl import load_workbook
wb = load_workbook(outputPath, data_only=True)
ws = wb.worksheets[1]

big_book_cols = newPrice_data.columns
last_col = len(df.columns)

from openpyxl.utils.cell import get_column_letter

pn_cell = '{}{}'.format(get_column_letter(last_col - 1), 5)
pricing_cell = '{}{}'.format(get_column_letter(last_col), 5)

In [15]:
ws[pn_cell].value = big_book_cols[0]
ws[pricing_cell].value = big_book_cols[1]
wb.save(outputPath)